<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Классификация токсичных комментариев с применением BERT

Нужно построить инструмент, который будет искать токсичные комментарии.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Для генерации фичей примененим BERT. Желательно достигнуть метрики F1 не меньше 0.75 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [166]:
# Импортирование необходимых модулей и атрибутов
import pandas as pd
import numpy as np
import os
import re
import nltk
import torch

from numba import cuda
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import transformers as ppb
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from tqdm import notebook
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score 
from sklearn.model_selection import cross_val_score
#from pymystem3 import Mystem

In [167]:
# Создадим вспомогательные переменные, константы
SEED = 12345

In [168]:
# Объявим функцию, которая будет читать файлы
def pth_load(pth1, pth2):
    """Sapport using os.path.exists. Load local file in primarily

    :param pth1: local addres of file
    :type pth1: object
    :param pth2: external addres of file
    :type pth2: object
    
    :raises ValueError: if file not found in addresses
    
    :rtype: DataFrame
    :return: foundly file in the form of DataFrame
    """
    if os.path.exists(pth1):
        df = pd.read_csv(pth1)
    elif os.path.exists(pth2):
        df = pd.read_csv(pth2)
    else:
        print('Something is wrong')
    return df

# Прочитаем файл, сохраним данные
df = pth_load('toxic_comments.csv', '/datasets/toxic_comments.csv')

In [169]:
# Вызовем метод 'info()' и напечатаем несколько строк. Прочитаем одну строку целиком
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [170]:
# Прочитаем одну строку целиком
df['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [171]:
# Напишем функцию для очистки текста
def clear_text(text):
    """performs cleans the text

    :param text: text to clear
    :type text: object
    
    :rtype: object
    :return: cleared text
    """
    text = re.sub(r'[^a-z\' ^0-9]', ' ', text)
    #text = re.sub(r'[^a-z]', ' ', text)
    clean_text = " ".join(text.split())
    
    return clean_text

In [172]:
# Переведем символы в нижний регистр
df['lemm_text'] = df['text'].str.lower()

# Обрежем текст ячеек от лишних символов
df['lemm_text'] = df['lemm_text'].apply(lambda x: clear_text(x))

#df['lemm_text'] = df['lemm_text'].str.slice(0, 512) # можно обрезать ячейки до 512 символов в случае необходимости

Чтобы ускорить лемматизацию, объединим ячейки и вызовем функцию лемматизации к объединенной строке. Проблема в том, что многие функции лемматизации, например Mystem, при каждом вызове загружается заново, сильно замедляя процесс. Также можно применить распараллеливание процесса.

В данном случае мы используем WordNetLemmatizer, работающий с английским языком, но в рамках учебного процесса оставим этот метод.
Данный метод был описан на странице https://itnan.ru/post.php?c=1&p=503420

In [173]:
# Сколько ячеек объединяем в одну
batch_size = 1000

text_batch = [list(df['lemm_text'])[i: i + batch_size] for i in range(0, len(df), batch_size)]

In [174]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [175]:
# Лемматизируем объединенные ячейки и разделим обратно на отдельные
def checkExecTimeMystemOneText(texts):
    """performs lemmatization the text. first it combines the text for joint processing, at the end it separates

    :param text: text to lemmatization
    :type text: object
    
    :rtype: object
    :return: lemmatized text
    """
    #m = Mystem()
    lemmatizer = WordNetLemmatizer()
    
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(texts, batch_size)
    res = []
    for txtp in txtpart:
        alltexts = ' '.join([txt + ' brblm ' for txt in txtp])
        
        #words = m.lemmatize(alltexts)
        words = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(alltexts)]
        
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != '':
                if txt == 'brblm':
                    res.append(doc)
                    doc = []
                else:
                    doc.append(txt)
                    
        return res

processed_texts = Parallel(n_jobs=-1)(delayed(checkExecTimeMystemOneText)(t) for t in tqdm(text_batch))

100%|██████████| 160/160 [00:40<00:00,  3.96it/s]


In [176]:
# Получился вложенный список. Развернем верхний слой и объединим список в текст
lemm_text = sum(processed_texts, [])

# Удалим использованные переменные
del text_batch
del processed_texts

for i in range(0, len(lemm_text), 1):
    lemm_text[i] = " ".join(lemm_text[i]).replace(' \' ','\'')

# Сохраним в набор данных
df['lemm_text'] = lemm_text
del lemm_text

# Проверим результат
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   lemm_text  159571 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i 'm really not trying to edit war it ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


До этого момента код был аналогичен проекту без использования берта

In [177]:
# Вычисления всего датасета даже с использованием видеокарты будет проводиться несколько часов, поэтому ограничим выборку
exampl_lemm = df.sample(50000)
exampl_lemm

,text,toxic,lemm_text
131014,"Just updated. SRS won 4, and not 5.",0,just updated sr won 4 and not 5
5442,"""\n\nThis article is being rewritten at Jack A...",0,this article is being rewritten at jack abramo...
83578,"Again, you don't decide what NPOV means! When ...",0,again you do n't decide what npov mean when yo...
47505,Listen... \n\n...I've been vandalising for eig...,0,listen i 've been vandalising for eight year i...
144099,My long-term memory may be failing me. It was ...,0,my long term memory may be failing me it wa me...
...,...,...,...
139899,which may contain more details,0,which may contain more detail
17760,that is so significant and of such consequence,0,that is so significant and of such consequence
127779,PS: in WP it is mentioned here: Idaho_National...,0,p in wp it is mentioned here idaho national la...
1134,Evan Rachel Wood\nIs there any evidence that s...,0,evan rachel wood is there any evidence that sh...


In [178]:
# Загрузка предобученной модели/токенизатора 
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [179]:
# Будем проводить вычисления на GPU
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [180]:
# Создаем токенайзеры и паддинги
tokenized = exampl_lemm['lemm_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, padding=True, truncation=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

padded = torch.tensor(padded).to(torch.int64)

In [181]:
# Генерируем фичи 
batch_size = 10 # для примера возьмем такой батч
embeddings = [] 
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda() # закидываем тензор на GPU
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

    with torch.no_grad():
        model.cuda()
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на CPU чтобы сохранить в numpy
    del batch
    del attention_mask_batch
    del batch_embeddings

features = np.concatenate(embeddings) 

# Проверим результат
features.shape

  0%|          | 0/5000 [00:00<?, ?it/s]

(50000, 768)

In [182]:
# Генерируем таргеты, т.е. целевые признаки
target = exampl_lemm['toxic']
target.shape

(50000,)

In [183]:
# Разделим полученные выборки на тренировочную и тестовую
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.2, 
                                                                            random_state=SEED
                                                                            )

Мы успешно получили тренировочные и тестовые фачи и целевые признаки, теперь можно обучить любую модель для генерации предсказания

## Обучение

Обучим модель логистической регрессии и посмотрим на метрику F1. Подбирать гиперпараметры не будем.

In [184]:
# Обучим модель и сгенерируем предсказания
logistic_model = LogisticRegression(random_state=SEED, max_iter=2000)
logistic_model.fit(features_train, target_train)
predictions = logistic_model.predict(features_test)
predictions = pd.Series(predictions, index = target_test.index)

In [185]:
# Посчитаем целевую метрику и выведем не экран
accuracy = f1_score(target_test, predictions)
print('Метрика f1: {:.3f}'.format(accuracy))

Метрика f1: 0.687


## Выводы

Мы использовали BERT для генерации фичей из лемматизированной строки, и моделью логистической регрессии предсказали принадлежность строки к одному из двух категорий: токсичная и нетоксичная. Модель на тестовой выбоке показала метрику F1 в 0.687. Неплохой результат, это больше, чем большинство моделей в проекте "Классификация токсичности комментариев" (без применения BERT).